In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import argparse
import numpy as np
import pandas as pd
import pickle
import os.path
import scipy.sparse as sp
import scipy.io
import os
# import PreprocessingscGNN
import h5py
from sklearn.manifold import TSNE
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, silhouette_score, calinski_harabasz_score
import glob2


In [3]:
for category in ["balanced_data", "imbalanced_data", "real_data", 
                ]:

    path= "../../"
    if category in ["balanced_data", "imbalanced_data"]:
        files = glob2.glob(f'{path}/R/simulated_data/{category}/*.h5')
        files = [f[len(f"{path}/R/simulated_data/{category}/"):-3] for f in files]
    else:
        files = glob2.glob(f'{path}/real_data/*.h5')
        files = [f[len(f"{path}/real_data/"):-3] for f in files]
    print(files)

    df = pd.DataFrame(columns = ["dataset", "ARI", "NMI", "sil", "run", "time", "pred", "cal", "features"])
    for dataset in files:
        if category in ["balanced_data", "imbalanced_data"]:
            data_mat = h5py.File(f"{path}/R/simulated_data/{category}/{dataset}.h5","r")
        else:
            data_mat = h5py.File(f"{path}/real_data/{dataset}.h5","r")
        for run in range(3):
            Y = np.array(data_mat['Y'])
            X = np.array(data_mat['X'])
            print(f">>>>dataset {dataset}")
            n_clusters =  np.unique(Y).shape[0]
            X = np.ceil(X).astype(np.int)
            
            # Create CSV input file, as required by scGNN
            df1 = pd.DataFrame(X.T)
            df1 = df1.reset_index(drop = True)
            df1.columns = np.arange(df1.shape[1]).astype(str)
            os.system("rm -r myres")
            os.system("mkdir myres")
            df1.to_csv("myres/expression.csv")
            start = time.time()
            command = ("python -W ignore PreprocessingscGNN.py --datasetName expression.csv " +
           "--datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile")

            del df1, X
            print(command)
            os.system(command)

            os.system("rm -r outputdir")
            command = "python -W ignore scGNN.py --datasetName myres --datasetDir ./  " 
            command += "--outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 "
            command += "--quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv "
            command += f" --seed {run} > logfile" 
            print(command)

            os.system(command)

            emb = pd.read_csv("outputdir/myres_embedding.csv")
            emb = emb.values[:, 1:]
            pred = pd.read_csv("outputdir/myres_results.txt")

            pred = pred.values[:, 1]

            elapsed = time.time() - start
            ARI = adjusted_rand_score(Y, pred)
            NMI = np.around(normalized_mutual_info_score(Y, pred), 5)
            ss = silhouette_score(emb,pred)
            cal = calinski_harabasz_score(emb, pred)
            emb = TSNE(n_components=2).fit_transform(emb)

            df.loc[df.shape[0]] = [
                dataset, ARI, NMI, ss, run, elapsed, pred, cal, emb
            ]
            df.to_pickle(f"../../output/pickle_results/{category}/{category}_scGNN.pkl")
            os.system("rm -r myres")
            os.system("rm -r outputdir")

['data_0c8', 'data_1c4', 'data_0c16', 'data_1.5c16', 'data_0c4', 'data_1.5c4', 'data_1.5c8', 'data_-1c16', 'data_1c16', 'data_1c8', 'data_-1c8', 'data_-1c4']
>>>>dataset data_0c8
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 0 > logfile
>>>>dataset data_0c8
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 1 > logfile
>>>>dataset da

python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 1 > logfile
>>>>dataset data_1.5c8
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 2 > logfile
>>>>dataset data_-1c16
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --ba

>>>>dataset data_1c4
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 0 > logfile
>>>>dataset data_1c4
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 1 > logfile
>>>>dataset data_1c4
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile


>>>>dataset data_-1c16
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 2 > logfile
>>>>dataset data_1c16
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 0 > logfile
>>>>dataset data_1c16
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logf

>>>>dataset worm_neuron_cell
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 0 > logfile
>>>>dataset worm_neuron_cell
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 1 > logfile
>>>>dataset worm_neuron_cell
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneS

>>>>dataset Adam
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 2 > logfile
>>>>dataset Quake_Smart-seq2_Trachea
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --geneSelectnum 2000 > logfile
python -W ignore scGNN.py --datasetName myres --datasetDir ./  --outputDir outputdir/ --EM-iteration 2 --Regu-epochs 500 --EM-epochs 20 --quickmode --nonsparseMode --batch-size 6000 --ltmgExpressionFile Use_expression.csv  --seed 0 > logfile
>>>>dataset Quake_Smart-seq2_Trachea
python -W ignore PreprocessingscGNN.py --datasetName expression.csv --datasetDir myres/ --LTMGDir myres/ --filetype CSV --g

In [4]:
df.mean()

ARI        0.396190
NMI        0.565139
sil        0.253787
time     142.989101
cal     1139.073544
dtype: float64